In [138]:
# predict HK stocks according to US stocks
    # there is jet-lag between US stocks and HK stocks
    # US stock markets have much influence over HK stock markets
    # exploit the information from US stock markets to predict HK stock markets

In [280]:
# import packages needed
import yfinance as yf #!pip install yfinance
import math
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split 
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [281]:
# get data by ticker-name, start-time & end-time
def get_df_data(ticker_name="AAPL", start_time="2022-01-01", end_time="2022-10-09"):
  df_data = yf.download(tickers=ticker_name, start=start_time, end=end_time) 
  #df_data.head()
  return df_data

# calculate the daily return by (current_index - previous_index) / previous_index
def calculate_daily_return(df_data, OHLC_index="Close"):
  name1 = OHLC_index+"_previous"
  df_data[name1] = df_data[OHLC_index].shift(1)
  name2 = OHLC_index+"_delta"
  df_data[name2] = df_data[OHLC_index] - df_data[name1]
  name3 = OHLC_index+"_return"
  df_data[name3] = df_data[name2] / df_data[name1]
  del df_data[name1]
  del df_data[name2]
  new_feature = name3
  return df_data #, new_feature

# calculate the daily change of points & volumes
    # by (current_index - previous_index) / previous_index
    # by (current_volume - previous_volume) / previous_volume
def calculate_daily_change(df_data):
  # , OHLC_index="Close"
  df_data = calculate_daily_return(df_data, "Close")
  df_data = calculate_daily_return(df_data, "Volume")
  return df_data #, new_feature

# convert the time to be string type: yyyy-mm-dd
def get_ymt_date(df_data):
  df_data["ymd_time"] = df_data.index
  df_data["ymd_time"] = df_data["ymd_time"].astype(str)
  df_data["ymd_time"] = df_data["ymd_time"].str.slice(0,10)
  return df_data

# dict type: key - time, value - daily return
def get_date_return(df_data):
  return_list = list(df_data["Close_return"])
  time_list = list(df_data["ymd_time"])
  time_return_dic = {}
  ii = 0
  while ii<len(return_list):
    time_return_dic[ time_list[ii] ] = return_list[ii]
    """if return_list[ii]>0:
      time_return_dic[ time_list[ii] ] = 1
    else:
      time_return_dic[ time_list[ii] ] = 0"""
    ii = ii + 1
  return time_return_dic

# get useful features and pack as dict-type
    # key: date, value:
def get_date_features(df_data, features=["Close_return", "Volume_return"]):
    time_list = list(df_data["ymd_time"])
    values_list = []
    L = len(df_data)
    for ii in range(L):
        fv = []
        for fn in features:
            v = df_data.iloc[ii][fn]
            fv.append(v)
        values_list.append( fv )
    time_return_dic = {}
    ii = 0
    while ii<L:
        time_return_dic[ time_list[ii] ] = values_list[ii]
        ii = ii + 1
    return time_return_dic

# compare 2 string-type dates, 
  # for example: '2022-01-01' -> 20220101, '2022-10-10' -> 20221010,
def compare_date_str(date_str1, date_str2):
  num1 = date_str1[0:4]+date_str1[5:7]+date_str1[8:10]
  num1 = int(num1)
  num2 = date_str2[0:4]+date_str2[5:7]+date_str2[8:10]
  num2 = int(num2)
  if num1>num2:
    return 1
  elif num1<num2:
    return -1
  return 0

# 3 US stock indexes: Nasdaq, DJI, SP500, 1 HK index: HSI
# get their data for certain year
# close points -> moving average close points, parameter: nn
# calculate the daily return based on the MA close points
# get the date-return dic
def get_useful_data(tn, st, et, nn, features=["Close_return", "Volume_return"]):
  stock_index = get_df_data(ticker_name=tn, start_time=st, end_time=et)
  stock_index['SMA'+str(nn)] = stock_index['Close'].rolling(nn).mean() # moving average, smoothening function
  stock_index['Close'] = stock_index['SMA'+str(nn)] # moving average, smoothening function
  del stock_index['SMA'+str(nn)] # 
  stock_index = calculate_daily_change(stock_index)
  stock_index = get_ymt_date(stock_index)
  time_return = get_date_features(stock_index, features)
  return stock_index, time_return

# match US stocks with HK stock by time, because there is jet-lag between HK time & US time
def US_HK_stock_signal(year = 2022, nn = 5):
  st, et = str(year)+"-01-01", str(year)+"-12-31"
  # ^IXIC, ^DJI, ^GSPC
  nasdaq_df, nasdaq_time_return = get_date_return_dic("^IXIC", st, et, nn)
  dowjones_df, dowjones_time_return = get_date_return_dic("^DJI", st, et, nn)
  sp500_df, sp500_time_return = get_date_return_dic("^GSPC", st, et, nn)

  # ^HSI
  hsi_df, hsi_time_return = get_date_return_dic("^HSI", st, et, nn)  
  
  # 
  hk_us_stock_signal = []

  for k, v in sorted(hsi_time_return.items()):
    date_str = k
    rise_fall_signal = v
    for pk, pv in sorted(sp500_time_return.items(), reverse=True):
      if compare_date_str(k, pk)==1:
        v1 = sp500_time_return[pk]
        v2 = dowjones_time_return[pk]
        v3 = nasdaq_time_return[pk]
        #print(k, v, pk, v1, v2, v3)
        hk_us_stock_signal.append( (k, v, pk, v1, v2, v3) )
        break
  return hk_us_stock_signal

# filter signals according to HK stock rise/fall
def US_HK_info(hk_us_stock_signal, hk_s, us_stock_num):
  tmp_list = []
  for it in hk_us_stock_signal:
    hkt, hk_signal, ust, us_signal1, us_signal2, us_signal3 = it
    hk_signal = rise_fall_judge(hk_signal)
    us_signal1 = rise_fall_judge(us_signal1)
    us_signal2 = rise_fall_judge(us_signal2)
    us_signal3 = rise_fall_judge(us_signal3)
    #
    if hk_signal==hk_s and sum( [us_signal1, us_signal2, us_signal3] ) == us_stock_num:
      tmp_list.append( it )
  return tmp_list

# rise -> 1, fall -> 0
def rise_fall_judge(point):
  if point > 0:
    return 1
  return 0

# match US stocks with HK stock by time, because there is jet-lag between HK time & US time
def stocks_signal(refer_list=["^IXIC", "^DJI", "^GSPC"], target="^HSI", st="2022-01-01", et="2022-12-31", nn=5, features=["Close_return", "Volume_return"]):
  # reference list
  refer_dic_list = []
  for it in refer_list:
    _, tmp = get_useful_data(it, st, et, nn, features)
    refer_dic_list.append( tmp )
  # target
  _, target_dic = get_useful_data(target, st, et, nn, features) # ["Close_return"]
  # integrate references & target
  hk_us_stock_signal = []
  for k, v in sorted(target_dic.items()):
    date_str = k
    rise_fall_signal = v
    for pk, pv in sorted(refer_dic_list[0].items(), reverse=True):
      if compare_date_str(k, pk)==1:
        vs = []
        for rd in refer_dic_list:
          vs.append( rd[pk] )
        #print(k, v, pk, vs)
        hk_us_stock_signal.append( (k, v, pk, vs) )
        break
  return hk_us_stock_signal

# list of lists -> dataframe
def lists_to_dataframe(hk_us_stock_signal, refer_list, target):
  tmp_list = []
  L = 0
  for it in hk_us_stock_signal[ : ]:
    hkt, hk_hsi, ust, vs = it
    # flatten list of lists to be list 
    vs = np.array(vs)
    vs = vs.reshape(-1)
    vs = list(vs)
    L = len(vs)
    sub_list = []
    sub_list.append( hkt )
    sub_list += hk_hsi
    sub_list.append( ust )
    sub_list += vs
    tmp_list.append( sub_list )
  cols = ["HKT"]
  cols.append( target+"_close" )
  cols.append( target+"_volume" )
  cols.append( "UST" )
  for it in refer_list:
    cols.append( it+"_close" )
    cols.append( it+"_volume" )
  #print( len(cols), len(tmp_list[0]) )
  assert len(cols)==len(tmp_list[0])
  df_data = pd.DataFrame(tmp_list, columns=cols)
  return df_data

# scale list
def scale_list_values(vs):
  tmp = []
  for v in vs:
    tmp.append( v*100 )
  return tmp

# get data-sets for training & testing models from dataframe
def dataframe_Xy(df_data):
    X, y = [], []
    for i in range(len(df_data)):
        row = list( df_data.iloc[i] )
        target_daily_return = row[1]
        refer_values = row[4:]
        #
        if math.isnan(target_daily_return):
            continue
        fg = False
        for v in refer_values:
            if math.isnan(v):
                fg = True
                break
        if fg:
            continue
        #
        X.append( scale_list_values(refer_values) )
        if target_daily_return>0:
            y.append(1)
        else:
            y.append(0)
    return X, y

# sample data for training & testing model
def sample_dataset(df_data, ran_seed, sample_number=-1):
    # balance the positive and negative samples
    positive_df = df_data[ df_data[target+"_close"]>0 ].copy()
    negative_df = df_data[ df_data[target+"_close"]<0 ].copy()
    positive_df = positive_df.dropna()
    negative_df = negative_df.dropna()
    print( "business days: ", len(df_data), "rise vs fall: ", len(positive_df), len(negative_df) )
    if sample_number==-1:
        sample_number = min( len(positive_df), len(negative_df) )
    X_pos_data, y_pos_data = dataframe_Xy(positive_df.sample(n=sample_number, random_state=ran_seed))
    X_neg_data, y_neg_data = dataframe_Xy(negative_df.sample(n=sample_number, random_state=ran_seed))
    X_data = X_pos_data + X_neg_data
    y_data = y_pos_data + y_neg_data
    print( "sampled business days: ", len(X_data), "rise vs fall: ", len(X_pos_data), len(X_neg_data) )
    return X_data, y_data

# train LR model
def train_LR_model(X_data, y_data, ran_seed):
    ### data-set split and train models
    X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.25, random_state=ran_seed, shuffle=True)
    print( "training data: ", len(X_train), len(y_train) )
    print( "testing data: ", len(X_test), len(y_test) )
    #
    # LR
    LR1 = LogisticRegression()
    LR1.fit(X_train, y_train)
    y_pred = LR1.predict(X_test)
    score = LR1.score(X_test, y_test)
    print(classification_report(y_test, y_pred))
    print("LR", score)
    return LR1, score

# train SVM model
def train_SVM_model(X_data, y_data, ran_seed):
    ### data-set split and train models
    X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.25, random_state=ran_seed, shuffle=True)
    print( "training data: ", len(X_train), len(y_train) )
    print( "testing data: ", len(X_test), len(y_test) )
    #
    # SVM
    SVM1 = svm.SVC(kernel='linear', probability=True)  # solver='lbfgs', , max_iter=1000 * 1000 * 20
    SVM1.fit(X_train, y_train)
    y_pred = SVM1.predict(X_test)
    score = SVM1.score(X_test, y_test)
    print(classification_report(y_test, y_pred))
    print("SVM", score)
    return SVM1, score

# train RF model
def train_RF_model(X_data, y_data, ran_seed):
    ### data-set split and train models
    X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.25, random_state=ran_seed, shuffle=True)
    print( "training data: ", len(X_train), len(y_train) )
    print( "testing data: ", len(X_test), len(y_test) )
    #
    # RF
    RF1=RandomForestClassifier(n_estimators=100)
    RF1.fit(X_train, y_train)
    y_pred = RF1.predict(X_test)
    score = RF1.score(X_test, y_test)
    print(classification_report(y_test, y_pred))
    print("RF", score)
    return RF1, score

In [282]:
### prepare data to fit models
nn = 1
dataset_xy = []
refer_list=["^IXIC", "AAPL", "GOOGL", "AMZN", "BABA", "PDD", "JD", "MPNGY", "TME", "BIDU"] # "^IXIC", "^DJI", "^GSPC"
target="9888.HK" # 0700.HK, ^HSI, 0005.HK:滙豐控股, 1299.HK:友邦保險, 0700.HK:騰訊控股, 9988.HK:阿⾥巴巴, 3690.HK:美團, '9888.HK':百度, 9866.HK:蔚来
#st, et = str(year)+"-01-01", str(year)+"-12-31"
st, et = "2022-01-01", "2022-11-16"
features=["Close_return", "Volume_return"]

hk_us_stock_signal = stocks_signal(refer_list, target, st, et, nn, features)
df_data = lists_to_dataframe(hk_us_stock_signal, refer_list, target)
df_data

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,HKT,9888.HK_close,9888.HK_volume,UST,^IXIC_close,^IXIC_volume,AAPL_close,AAPL_volume,GOOGL_close,GOOGL_volume,...,PDD_close,PDD_volume,JD_close,JD_volume,MPNGY_close,MPNGY_volume,TME_close,TME_volume,BIDU_close,BIDU_volume
0,2022-01-04,0.009053,1.356869,2022-01-03,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-01-05,-0.040028,0.397103,2022-01-04,-0.013269,0.158275,-0.012692,-0.049551,-0.004083,-0.008588,...,-0.111943,1.359285,-0.060442,1.132382,-0.061696,1.729614,-0.058394,0.183676,-0.017237,0.235208
2,2022-01-06,0.020848,-0.449717,2022-01-05,-0.033448,-0.019345,-0.026600,-0.048059,-0.045876,0.923169,...,-0.025893,-0.410963,-0.018380,-0.040105,-0.074458,0.207547,-0.017054,-0.135264,-0.018085,0.218680
3,2022-01-07,0.049296,0.475324,2022-01-06,-0.001279,-0.047901,-0.016693,0.025031,-0.000200,-0.316196,...,0.074181,0.505374,0.059664,-0.324300,0.051031,-0.020833,0.039432,0.299012,0.047748,0.095011
4,2022-01-10,0.018792,-0.170949,2022-01-07,-0.009612,-0.115377,0.000988,-0.105206,-0.005303,-0.203170,...,0.072319,-0.138743,0.021863,0.003309,0.012376,-0.564495,0.024279,-0.158115,0.017114,-0.283147
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209,2022-11-09,-0.040646,0.342924,2022-11-08,0.004892,0.152088,0.004175,0.078368,0.004633,-0.107101,...,0.019443,-0.377069,0.022018,-0.097408,-0.012503,-0.205364,-0.020151,0.498235,0.002000,-0.334956
210,2022-11-10,-0.024299,-0.328610,2022-11-09,-0.024776,-0.030585,-0.033190,-0.166733,-0.017773,0.044040,...,-0.059758,-0.109792,-0.066608,0.019527,-0.067001,-0.201543,-0.059126,0.179761,-0.067027,0.574070
211,2022-11-11,0.091315,2.042460,2022-11-10,0.073502,0.292426,0.088975,0.586459,0.075813,0.624852,...,0.082657,0.334922,0.084079,0.255100,0.104043,0.836957,0.057377,0.107699,0.083417,0.137400
212,2022-11-14,0.038034,-0.260532,2022-11-11,0.018821,-0.087867,0.019269,-0.209923,0.026293,-0.359310,...,0.017642,0.049786,0.063002,0.081773,0.061716,-0.566075,0.139535,0.137444,0.038904,-0.009749


In [284]:
fg = True # False, True
if fg:
    score_model = {'LR':[], 'SVM':[], 'RF':[]}
    for ran_seed1 in range(10):
        sample_number = -1
        X_data, y_data = sample_dataset(df_data, ran_seed1, sample_number)
        for ran_seed2 in range(10):
            LR1, score = train_LR_model(X_data, y_data, ran_seed2)
            score_model['LR'].append( (score, ran_seed1, ran_seed2) )
            
            SVM1, score = train_SVM_model(X_data, y_data, ran_seed2)
            score_model['SVM'].append( (score, ran_seed1, ran_seed2) )
            
            RF1, score = train_RF_model(X_data, y_data, ran_seed2)
            score_model['RF'].append( (score, ran_seed1, ran_seed2) )
            print( ran_seed1, ran_seed2 )
    #
    LR_list = sorted(score_model['LR'])
    print(LR_list[-10:])
    SVM_list = sorted(score_model['SVM'])
    print(SVM_list[-10:])
    RF_list = sorted(score_model['RF'])
    print(RF_list[-10:])



business days:  214 rise vs fall:  100 113
sampled business days:  200 rise vs fall:  100 100
training data:  150 150
testing data:  50 50
              precision    recall  f1-score   support

           0       0.68      0.63      0.65        27
           1       0.60      0.65      0.63        23

    accuracy                           0.64        50
   macro avg       0.64      0.64      0.64        50
weighted avg       0.64      0.64      0.64        50

LR 0.64
training data:  150 150
testing data:  50 50


C:\Users\Admin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


              precision    recall  f1-score   support

           0       0.67      0.67      0.67        27
           1       0.61      0.61      0.61        23

    accuracy                           0.64        50
   macro avg       0.64      0.64      0.64        50
weighted avg       0.64      0.64      0.64        50

SVM 0.64
training data:  150 150
testing data:  50 50
              precision    recall  f1-score   support

           0       0.65      0.63      0.64        27
           1       0.58      0.61      0.60        23

    accuracy                           0.62        50
   macro avg       0.62      0.62      0.62        50
weighted avg       0.62      0.62      0.62        50

RF 0.62
0 0
training data:  150 150
testing data:  50 50
              precision    recall  f1-score   support

           0       0.69      0.78      0.73        23
           1       0.79      0.70      0.75        27

    accuracy                           0.74        50
   macro avg     

C:\Users\Admin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


              precision    recall  f1-score   support

           0       0.71      0.74      0.72        23
           1       0.77      0.74      0.75        27

    accuracy                           0.74        50
   macro avg       0.74      0.74      0.74        50
weighted avg       0.74      0.74      0.74        50

SVM 0.74
training data:  150 150
testing data:  50 50
              precision    recall  f1-score   support

           0       0.68      0.83      0.75        23
           1       0.82      0.67      0.73        27

    accuracy                           0.74        50
   macro avg       0.75      0.75      0.74        50
weighted avg       0.75      0.74      0.74        50

RF 0.74
0 1
training data:  150 150
testing data:  50 50
              precision    recall  f1-score   support

           0       0.70      0.79      0.75        24
           1       0.78      0.69      0.73        26

    accuracy                           0.74        50
   macro avg     

...

              precision    recall  f1-score   support

           0       0.80      0.71      0.75        28
           1       0.68      0.77      0.72        22

    accuracy                           0.74        50
   macro avg       0.74      0.74      0.74        50
weighted avg       0.75      0.74      0.74        50

SVM 0.74
training data:  150 150
testing data:  50 50
              precision    recall  f1-score   support

           0       0.82      0.64      0.72        28
           1       0.64      0.82      0.72        22

    accuracy                           0.72        50
   macro avg       0.73      0.73      0.72        50
weighted avg       0.74      0.72      0.72        50

RF 0.72
9 7
training data:  150 150
testing data:  50 50
              precision    recall  f1-score   support

           0       0.78      0.72      0.75        29
           1       0.65      0.71      0.68        21

    accuracy                           0.72        50
   macro avg     

C:\Users\Admin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


              precision    recall  f1-score   support

           0       0.81      0.76      0.79        29
           1       0.70      0.76      0.73        21

    accuracy                           0.76        50
   macro avg       0.76      0.76      0.76        50
weighted avg       0.76      0.76      0.76        50

SVM 0.76
training data:  150 150
testing data:  50 50
              precision    recall  f1-score   support

           0       0.91      0.72      0.81        29
           1       0.70      0.90      0.79        21

    accuracy                           0.80        50
   macro avg       0.81      0.81      0.80        50
weighted avg       0.83      0.80      0.80        50

RF 0.8
9 8
training data:  150 150
testing data:  50 50
              precision    recall  f1-score   support

           0       0.71      0.77      0.74        22
           1       0.81      0.75      0.78        28

    accuracy                           0.76        50
   macro avg      

C:\Users\Admin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


              precision    recall  f1-score   support

           0       0.76      0.73      0.74        22
           1       0.79      0.82      0.81        28

    accuracy                           0.78        50
   macro avg       0.78      0.77      0.78        50
weighted avg       0.78      0.78      0.78        50

SVM 0.78
training data:  150 150
testing data:  50 50
              precision    recall  f1-score   support

           0       0.77      0.77      0.77        22
           1       0.82      0.82      0.82        28

    accuracy                           0.80        50
   macro avg       0.80      0.80      0.80        50
weighted avg       0.80      0.80      0.80        50

RF 0.8
9 9
[(0.78, 7, 8), (0.78, 8, 1), (0.78, 8, 6), (0.8, 1, 6), (0.8, 5, 6), (0.8, 5, 8), (0.84, 8, 3), (0.86, 3, 2), (0.88, 0, 5), (0.9, 8, 4)]
[(0.8, 5, 3), (0.8, 6, 8), (0.8, 8, 1), (0.8, 8, 6), (0.82, 1, 6), (0.82, 5, 6), (0.82, 8, 3), (0.84, 0, 5), (0.84, 3, 2), (0.9, 8, 4)]
[(0.78, 

In [ ]:
#
ran_seed1, ran_seed2 = 1, 8
X_data, y_data = sample_dataset(df_data, ran_seed1, sample_number)
LR1, score = train_LR_model(X_data, y_data, ran_seed2)

ran_seed1, ran_seed2 = 6, 1
X_data, y_data = sample_dataset(df_data, ran_seed1, sample_number)
SVM1, score = train_SVM_model(X_data, y_data, ran_seed2)

ran_seed1, ran_seed2 = 9, 7
X_data, y_data = sample_dataset(df_data, ran_seed1, sample_number)
RF1, score = train_RF_model(X_data, y_data, ran_seed2)

In [279]:
# get latest US stock data to predict HK stock
df_reference = pd.DataFrame()
year = 2022
st, et = str(year)+"-01-01", str(year)+"-12-31"
#refer_list=["^IXIC", "AAPL", "GOOGL", "AMZN", "BABA", "PDD", "JD", "MPNGY", "TME", "BIDU"] # "^IXIC", "^DJI", "^GSPC"
#target="9988.HK" # 0700.HK, ^HSI, 0005.HK:滙豐控股, 1299.HK:友邦保險, 0700.HK:騰訊控股, 9988.HK:阿⾥巴巴, 3690.HK:美團
lastest_data = []
lastest_day = -1
for tn in refer_list:
    tmp_df = get_df_data(tn, st, et)
    tmp_df = calculate_daily_change(tmp_df)
    #tmp_df = tmp_df[ ['Close', 'Close_return'] ]
    #df_reference[tn+"_Close"] = tmp_df[ 'Close' ]
    df_reference[tn+"_Close_return"] = tmp_df[ 'Close_return' ]
    df_reference[tn+"_Volume_return"] = tmp_df[ 'Volume_return' ]
    date_time = list(tmp_df.index)[lastest_day]
    close_point = list(tmp_df[ 'Close_return' ])[lastest_day]
    volume_point = list(tmp_df[ 'Close_return' ])[lastest_day]
    lastest_data.append( close_point )
    lastest_data.append( volume_point )
    print("date_time:", date_time, tn, close_point*100, volume_point*100)
df_reference
lastest_data = scale_list_values(lastest_data) # scale the data as the procedure of training data
print("lastest_data of US stock market info: ", lastest_data)

#
rise_or_fall = LR1.predict([lastest_data])
prob = LR1.predict_proba([lastest_data])
print( "LR predict today: ", rise_or_fall, prob )

rise_or_fall = SVM1.predict([lastest_data])
prob = SVM1.predict_proba([lastest_data])
print( "SVM predict today: ", rise_or_fall, prob )

rise_or_fall = RM1.predict([lastest_data])
prob = RM1.predict_proba([lastest_data])
print( "RF predict today: ", rise_or_fall, prob )

[*********************100%***********************]  1 of 1 completed
date_time: 2022-11-15 00:00:00 ^IXIC 1.4486177803630533 1.4486177803630533
[*********************100%***********************]  1 of 1 completed
date_time: 2022-11-15 00:00:00 AAPL 1.1869399253607704 1.1869399253607704
[*********************100%***********************]  1 of 1 completed
date_time: 2022-11-15 00:00:00 GOOGL 2.8631197288814447 2.8631197288814447
[*********************100%***********************]  1 of 1 completed
date_time: 2022-11-15 00:00:00 AMZN 0.45690383531042505 0.45690383531042505
[*********************100%***********************]  1 of 1 completed
date_time: 2022-11-15 00:00:00 BABA 11.173420743181945 11.173420743181945
[*********************100%***********************]  1 of 1 completed
date_time: 2022-11-15 00:00:00 PDD 8.533417775100988 8.533417775100988
[*********************100%***********************]  1 of 1 completed
date_time: 2022-11-15 00:00:00 JD 7.138645382580763 7.138645382580763
[*